In [1]:
import psycopg2
import configparser
import yfinance as yf
import pandas as pd

In [2]:
def create_connection():
    conn = psycopg2.connect("host=127.0.0.1 dbname=dmqldb user=postgres password=student")
    cur = conn.cursor()       
    return conn, cur

conn, cur = create_connection()

In [3]:
def load_data(conn, df, query):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]  
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [4]:
df_market = pd.read_csv('Market.csv')
market_insert_sql = """ INSERT INTO market VALUES (%s, %s, %s, %s) """
load_data(conn, df_market, market_insert_sql)

execute_many() done


In [5]:
symbols = ['MSFT', 'GOOGL']

In [6]:
for symbol in symbols:
    msft = yf.Ticker(symbol)
    list_msft = msft.info
    list_msft = pd.DataFrame([list_msft])
    df_company = pd.DataFrame()
    df_company['Name'] = list_msft['longName']
    df_company['Sector']= list_msft['sector']
    df_company.insert(0, "Symbol", symbol, True)
    company_insert_sql = """ INSERT INTO company VALUES (%s, %s, %s) """
    load_data(conn, df_company, company_insert_sql)

execute_many() done
execute_many() done


In [7]:
for symbol in symbols:
    msft = yf.Ticker(symbol)
    df_hist = msft.history(period='max')
    df_hist.reset_index(level=0, inplace=True)
    df_hist.insert(0, "Symbol", symbol, True)
    df_hist.insert(0, "Market", 'NYSE', True)
    hist_insert_sql = """ INSERT INTO stock_history VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """
    load_data(conn, df_hist, hist_insert_sql)

execute_many() done
execute_many() done


In [8]:
conn.close()